In [15]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

import warehouse_target
import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = 'ra-development'

sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.lookml_explore_schema

if warehouse_name == 'big_query':

    # Run a Standard SQL query with the project set explicitly
    project_id = lookml_project
    df = pandas.read_gbq(sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    df['description'] = df['description'].fillna('not available')

    df1 = df[['table_name','column_name','data_type','description']]

    df1['data_type'] = df1['data_type'].str.replace('TIMESTAMP','timestamp')
    df1['data_type'] = df1['data_type'].str.replace('DATE','date')
    df1['data_type'] = df1['data_type'].str.replace('INT64','number')
    df1['data_type'] = df1['data_type'].str.replace('FLOAT64','number')
    df1['data_type'] = df1['data_type'].str.replace('NUMERIC','number')
    df1['data_type'] = df1['data_type'].str.replace('STRING','string')
    df1['data_type'] = df1['data_type'].str.replace('BOOL','yesno')

    df2 = df1

    explore_df = pandas.read_gbq(explore_sql, dialect='standard', project_id=lookml_project, credentials=credentials)
   

elif warehouse_name == 'snowflake': 

    url = URL(

        account = config.snowflake_account,
        user =  config.snowflake_user,
        schema =  config.snowflake_schema,
        database =  config.snowflake_database,
        password =  config.snowflake_password,
        warehouse= config.snowflake_warehouse,
        role =  config.snowflake_role

    )

    engine = create_engine(url)

    connection = engine.connect()

    query = '''
    select * from snowflake_sample_data.information_schema.columns;
    '''

    df = pd.read_sql(query, connection)
    
    df['description'] = df['comment'].fillna('not available')
    
    df1 = df.groupby(['table_name', 'column_name','data_type','description']).size().reset_index().rename(columns={0:'count'})

    df2 = df1[['table_name','column_name','data_type','description']]

    connection.close()
    engine.dispose()

    
df3 = {n: grp.loc[n].to_dict('index')
    
for n, grp in df2.set_index(['table_name', 'column_name','data_type','description']).groupby(level='table_name')}

d1 = df3

##

df4 = {n: grp.loc[n].to_dict('index')
    
for n, grp in explore_df.set_index(['pk_table_name', 'pk_column_name','fk_table_name','fk_column_name']).groupby(level='pk_table_name')}

d2 = df4

				

GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ra-development/jobs?prettyPrint=false: Access Denied: Project ra-development: User does not have bigquery.jobs.create permission in project ra-development.

Location: None
Job ID: e8b05aa7-aaf1-4237-8643-e376e7391715


In [2]:
explore_df

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,client-kaplan-poc,analytics_qa,sales_quota_allocations,quota_allocation_pk,quota_allocation_pk,STRING,None
1,client-kaplan-poc,analytics_qa,sales_quota_allocations,university_code,university_code,STRING,None
2,client-kaplan-poc,analytics_qa,sales_quota_allocations,university_name,university_name,STRING,None
3,client-kaplan-poc,analytics_qa,sales_quota_allocations,college_code,college_code,STRING,None
4,client-kaplan-poc,analytics_qa,sales_quota_allocations,degree_concat,degree_concat,STRING,None
...,...,...,...,...,...,...,...
434,client-kaplan-poc,analytics_qa,targets_sales_regions,country_name,country_name,STRING,The target country where the student will star...
435,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_region,sales_region,STRING,The target sales region where the student will...
436,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_region_group,sales_region_group,STRING,The target sales region group where the studen...
437,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_channel,sales_channel,STRING,The target sales channel the student when thro...


In [8]:
explore_df

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,client-kaplan-poc,analytics_qa,sales_quota_allocations,quota_allocation_pk,quota_allocation_pk,STRING,None
1,client-kaplan-poc,analytics_qa,sales_quota_allocations,university_code,university_code,STRING,None
2,client-kaplan-poc,analytics_qa,sales_quota_allocations,university_name,university_name,STRING,None
3,client-kaplan-poc,analytics_qa,sales_quota_allocations,college_code,college_code,STRING,None
4,client-kaplan-poc,analytics_qa,sales_quota_allocations,degree_concat,degree_concat,STRING,None
...,...,...,...,...,...,...,...
434,client-kaplan-poc,analytics_qa,targets_sales_regions,country_name,country_name,STRING,The target country where the student will star...
435,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_region,sales_region,STRING,The target sales region where the student will...
436,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_region_group,sales_region_group,STRING,The target sales region group where the studen...
437,client-kaplan-poc,analytics_qa,targets_sales_regions,sales_channel,sales_channel,STRING,The target sales channel the student when thro...


In [9]:
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

import warehouse_target
import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.lookml_explore_schema


In [10]:
explore_sql

'\n\n\n                with source as (\n\n                select * from `ra-development.jordan_analytics_dev.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`\n                \n                where table_name in (\'wh_website_event_pages_fct\',\'wh_website_event_tracks_fct\',\'wh_website_users_dim\')\n\n                ),\n                \n                pks as (\n                    select \n                    table_name as pk_table_name,\n                    column_name as pk_column_name,\n                    trim(column_name, "_pk") as pk_sk,\n                    from source\n                    where column_name like \'%pk%\'\n                    ),\n\n                    fks as (\n                    select\n                    table_name as fk_table_name,\n                    column_name as fk_column_name,\n                    trim(column_name, "_fk") as fk_sk,\n                    from source\n                    where column_name like \'%fk%\'\n\n                    )\n\n           

In [12]:
project_id

'client-kaplan-poc'

In [13]:
lookml_project

'client-kaplan-poc'

In [14]:
lookml_project

'client-kaplan-poc'

In [16]:
lookml_project

'ra-development'